In [8]:
from google.colab import files
uploaded = files.upload()


Saving Trimet_Data.csv to Trimet_Data (1).csv


In [21]:
import pandas as pd
data  = pd.read_csv('Trimet_Data.csv')
print(data.head())

   EVENT_NO_TRIP  EVENT_NO_STOP            OPD_DATE  VEHICLE_ID  METERS  \
0      259172515      259172517  15FEB2023:00:00:00        4223      40   
1      259172515      259172517  15FEB2023:00:00:00        4223      48   
2      259172515      259172517  15FEB2023:00:00:00        4223      57   
3      259172515      259172517  15FEB2023:00:00:00        4223      73   
4      259172515      259172517  15FEB2023:00:00:00        4223     112   

   ACT_TIME  GPS_LONGITUDE  GPS_LATITUDE  GPS_SATELLITES  GPS_HDOP  
0     20469    -122.648137     45.493082              12       0.7  
1     20474    -122.648240     45.493070              12       0.8  
2     20479    -122.648352     45.493123              12       0.8  
3     20484    -122.648385     45.493262              12       0.7  
4     20489    -122.648347     45.493582              12       0.8  


In [27]:
#Dropping Unnecessary columns
#New Data
data_cleaned = data.drop(columns=['EVENT_NO_STOP', 'GPS_SATELLITES', 'GPS_HDOP'])
print(data_cleaned.head())


   EVENT_NO_TRIP            OPD_DATE  VEHICLE_ID  METERS  ACT_TIME  \
0      259172515  15FEB2023:00:00:00        4223      40     20469   
1      259172515  15FEB2023:00:00:00        4223      48     20474   
2      259172515  15FEB2023:00:00:00        4223      57     20479   
3      259172515  15FEB2023:00:00:00        4223      73     20484   
4      259172515  15FEB2023:00:00:00        4223     112     20489   

   GPS_LONGITUDE  GPS_LATITUDE  
0    -122.648137     45.493082  
1    -122.648240     45.493070  
2    -122.648352     45.493123  
3    -122.648385     45.493262  
4    -122.648347     45.493582  


In [31]:
# Define a function to determine which columns to load
def columns_to_use(cols):
    columns_to_exclude = ['EVENT_NO_STOP', 'GPS_SATELLITES', 'GPS_HDOP']
    return [col for col in cols if col not in columns_to_exclude]

# Read the CSV while filtering columns
data_filtered = pd.read_csv('Trimet_Data.csv', usecols=lambda column : column not in ['EVENT_NO_STOP', 'GPS_SATELLITES', 'GPS_HDOP'])

print(data_filtered.head())


   EVENT_NO_TRIP            OPD_DATE  VEHICLE_ID  METERS  ACT_TIME  \
0      259172515  15FEB2023:00:00:00        4223      40     20469   
1      259172515  15FEB2023:00:00:00        4223      48     20474   
2      259172515  15FEB2023:00:00:00        4223      57     20479   
3      259172515  15FEB2023:00:00:00        4223      73     20484   
4      259172515  15FEB2023:00:00:00        4223     112     20489   

   GPS_LONGITUDE  GPS_LATITUDE  
0    -122.648137     45.493082  
1    -122.648240     45.493070  
2    -122.648352     45.493123  
3    -122.648385     45.493262  
4    -122.648347     45.493582  


In [46]:

from datetime import datetime, timedelta

#converting OPD_DATE and ACT_TIME into a single TIMESTAMP
def create_timestamp(row):
    # Parse the date with the correct format
    opd_date = datetime.strptime(row['OPD_DATE'], '%d%b%Y:%H:%M:%S')
    # Create a timedelta from ACT_TIME
    act_time_delta = timedelta(seconds=int(row['ACT_TIME']))
    # Return the new timestamp
    return opd_date + act_time_delta

# Apply the function to each row to create the new TIMESTAMP column
data['TIMESTAMP'] = data.apply(create_timestamp, axis=1)
print(data['TIMESTAMP'].head(10))

KeyError: 'OPD_DATE'

In [45]:
# Drop the OPD_DATE and ACT_TIME columns
data = data.drop(columns=['OPD_DATE', 'ACT_TIME'])

KeyError: "['OPD_DATE', 'ACT_TIME'] not found in axis"

In [47]:
print(data.head(10))

   EVENT_NO_TRIP  EVENT_NO_STOP  VEHICLE_ID  METERS  GPS_LONGITUDE  \
0      259172515      259172517        4223      40    -122.648137   
1      259172515      259172517        4223      48    -122.648240   
2      259172515      259172517        4223      57    -122.648352   
3      259172515      259172517        4223      73    -122.648385   
4      259172515      259172517        4223     112    -122.648347   
5      259172515      259172517        4223     159    -122.648357   
6      259172515      259172517        4223     215    -122.648383   
7      259172515      259172517        4223     272    -122.648375   
8      259172515      259172517        4223     330    -122.648330   
9      259172515      259172517        4223     391    -122.648213   

   GPS_LATITUDE  GPS_SATELLITES  GPS_HDOP           TIMESTAMP  
0     45.493082              12       0.7 2023-02-15 05:41:09  
1     45.493070              12       0.8 2023-02-15 05:41:14  
2     45.493123              12      

In [48]:


# Calculating the differences in METERS and TIMESTAMP
data['dMETERS'] = data['METERS'].diff()
data['dTIMESTAMP'] = data['TIMESTAMP'].diff().dt.total_seconds()  # Converted timedelta to total seconds

# Calculate SPEED as meters per second
data['SPEED'] = data.apply(lambda row: row['dMETERS'] / row['dTIMESTAMP'] if row['dTIMESTAMP'] != 0 else 0, axis=1)

# Drop the temporary columns
data.drop(columns=['dMETERS', 'dTIMESTAMP'], inplace=True)

# Calculate the minimum, maximum, and average speed
speed_statistics = data['SPEED'].describe()

# Display the statistics
print(speed_statistics)


count    160.000000
mean       7.227206
std        4.420604
min        0.000000
25%        3.800000
50%        6.400000
75%       10.850000
max       17.400000
Name: SPEED, dtype: float64
